In [209]:
import pandas as pd
import requests
from tqdm import tqdm

In [188]:
r = requests.get('https://api.github.com/repos/COVIDExposureIndices/COVIDExposureIndices/git/trees/master?recursive=1')
fileList = pd.DataFrame(r.json()['tree'])
countyFiles = list(fileList[fileList.path.str.contains('lex_data/county_lex')].path)
stateFiles = list(fileList[fileList.path.str.contains('lex_data/state_lex')].path)

In [189]:
df = pd.read_csv('https://github.com/COVIDExposureIndices/COVIDExposureIndices/blob/master/lex_data/county_lex_2020-01-20.csv.gz?raw=true', compression='gzip')
df.columns = [df.columns[0]] + [f'{int(col)}' for col in df.columns[1:]]
df['COUNTY_PRE'] = df['COUNTY_PRE'].astype(str)

In [190]:
df.head()

,COUNTY_PRE,1001,1003,1005,1007,1009,1013,1015,1017,1019,...,56001,56005,56013,56021,56025,56029,56033,56037,56039,56041
0,1001,0.904768,0.008013,0.010658,0.050045,0.008863,0.073306,0.005619,0.015366,0.004742,...,0.000000,0.000275,0.0,0.000114,0.000000,0.0,0.000000,0.000521,0.0,0.000486
1,1003,0.017834,0.937736,0.010049,0.009830,0.006956,0.098893,0.004185,0.008290,0.003952,...,0.002191,0.001099,0.0,0.000913,0.000178,0.0,0.000461,0.002082,0.0,0.004375
2,1005,0.006539,0.000985,0.857795,0.003277,0.001907,0.009336,0.002092,0.011929,0.001844,...,0.000000,0.000275,0.0,0.000000,0.000000,0.0,0.000000,0.000260,0.0,0.000486
3,1007,0.019260,0.001543,0.003959,0.890080,0.006732,0.006570,0.002869,0.002022,0.002107,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,1009,0.006182,0.002594,0.003350,0.015192,0.919444,0.005878,0.019249,0.003841,0.027397,...,0.000365,0.000275,0.0,0.000000,0.000356,0.0,0.000000,0.000260,0.0,0.000000


In [191]:
validColumns = {}
geoidList = list(df.columns[1:])
for geoid in geoidList:
    tempList = geoidList.copy()
    tempList.remove(geoid)
    validColumns[geoid] = tempList

In [192]:
sumDf = pd.DataFrame(df.sum()).drop_duplicates().reset_index().drop(0)
sumDf.columns= ['COUNTY_PRE','SUM_IN']
sumDf = sumDf.merge(df, on="COUNTY_PRE")

In [193]:
df['SUM_OUT'] = df.apply(lambda x: sum(x[validColumns[x.COUNTY_PRE]]), axis=1)
df['SUM_IN'] = sumDf.apply(lambda x: x.SUM_IN - x[x.COUNTY_PRE], axis=1)

In [194]:
combinedOut = pd.DataFrame(df[['COUNTY_PRE','SUM_OUT']])
combinedOut.columns = ['GEOID', '2020-01-20']
combinedIn = pd.DataFrame(df[['COUNTY_PRE','SUM_IN']])
combinedIn.columns = ['GEOID', '2020-01-20']

# loop through the rest...

In [203]:
def getParseLexData(date):
    df = pd.read_csv(f'https://github.com/COVIDExposureIndices/COVIDExposureIndices/blob/master/lex_data/county_lex_{date}.csv.gz?raw=true', compression='gzip')
    df.columns = [df.columns[0]] + [f'{int(col)}' for col in df.columns[1:]]
    df['COUNTY_PRE'] = df['COUNTY_PRE'].astype(str)
    
    validColumns = {}
    geoidList = list(df.columns[1:])
    for geoid in geoidList:
        tempList = geoidList.copy()
        tempList.remove(geoid)
        validColumns[geoid] = tempList
        
    sumDf = pd.DataFrame(df.sum()).drop_duplicates().reset_index().drop(0)
    sumDf.columns= ['COUNTY_PRE','SUM_IN']
    sumDf = sumDf.merge(df, on="COUNTY_PRE")
    
    df['SUM_OUT'] = df.apply(lambda x: sum(x[validColumns[x.COUNTY_PRE]]), axis=1)
    df['SUM_IN'] = sumDf.apply(lambda x: x.SUM_IN - x[x.COUNTY_PRE], axis=1)
    
    outDf = df[['COUNTY_PRE', 'SUM_OUT']]
    outDf.columns = ['GEOID',date]
    inDf = df[['COUNTY_PRE', 'SUM_IN']]
    inDf.columns = ['GEOID',date]
    
    return {
        'in': outDf,
        'out': inDf
    }

In [212]:
combinedOut = pd.read_csv('./county_LEX_out.csv')
combinedIn = pd.read_csv('./county_LEX_in.csv')

for file in tqdm(countyFiles):
    fileDate = file[-17:-7]
    if fileDate in combinedOut.columns:
        continue
    data = getParseLexData(fileDate)
    combinedOut = combinedOut.merge(data['out'], on="GEOID", how="left")
    combinedIn = combinedIn.merge(data['in'], on="GEOID", how="left")

combinedOut.to_csv('./county_LEX_out.csv', index=False)
combinedIn.to_csv('./county_LEX_in.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 255/255 [06:14<00:00,  1.47s/it]


In [211]:
combinedOut

,GEOID,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,...,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16
0,1001,2.506744,3.428945,3.414465,3.341262,3.380751,3.258600,3.309018,3.444757,3.503031,...,3.658036,3.712643,3.704992,3.683715,3.760518,3.630181,3.637882,3.581939,3.609567,3.635531
1,1003,4.688605,2.052634,2.049915,2.058592,2.111402,2.106043,2.129860,2.079366,2.095283,...,2.943148,2.948554,3.019620,3.018574,3.169607,3.086914,3.006516,3.029239,3.046110,3.062760
2,1005,1.542556,3.378974,3.335586,3.474973,3.328486,3.216789,3.280003,3.428727,3.437374,...,4.007143,4.116610,4.066454,4.005278,4.438012,4.058423,3.960661,3.754706,3.827960,4.151004
3,1007,0.997290,3.250589,3.301078,3.245064,3.260202,3.039772,3.114270,3.364850,3.323894,...,3.684648,3.648777,3.757156,3.608169,3.565811,3.540318,3.736048,3.782418,3.928143,3.688419
4,1009,1.930568,3.225423,3.215836,3.180233,3.224137,3.120370,3.169213,3.262870,3.227452,...,3.716202,3.592903,3.689949,3.687377,3.606169,3.645709,3.643773,3.651280,3.692494,3.690304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,56029,0.247800,1.229864,1.196747,1.276000,1.159868,1.219118,1.130378,1.333460,1.276669,...,2.991949,3.193101,3.068202,3.106682,3.082638,2.979610,3.098076,3.103877,3.286186,3.053500
2014,56033,0.518350,1.723914,1.525536,1.648905,1.525331,1.605622,1.539203,1.404206,1.507121,...,1.992009,2.039939,2.278749,2.272333,2.695145,2.303859,2.286696,2.164621,2.043430,1.976555
2015,56037,3.637412,7.781225,8.835947,8.148573,7.697965,9.015662,8.501389,6.504086,7.490570,...,6.993556,7.839169,7.690117,7.673303,9.176727,8.658344,6.785326,6.942312,7.813361,7.741056
2016,56039,0.386815,1.985031,1.992935,2.023134,2.139789,2.302265,2.231262,2.180687,2.135411,...,4.398658,4.457803,4.505818,4.462985,4.557823,4.520896,4.521163,4.727856,4.528587,4.684354
